## एम्बेडिङहरू

हाम्रो अघिल्लो उदाहरणमा, हामी उच्च-आयामीय bag-of-words भेक्टरहरूमा काम गर्यौं जसको लम्बाइ `vocab_size` थियो, र हामीले कम-आयामीय positional representation भेक्टरहरूलाई sparse one-hot representation मा स्पष्ट रूपमा परिवर्तन गर्यौं। यो one-hot representation मेमोरी-प्रभावकारी छैन। साथै, प्रत्येक शब्दलाई एक-अर्काबाट स्वतन्त्र रूपमा व्यवहार गरिन्छ, जसले गर्दा one-hot encoded भेक्टरहरूले शब्दहरू बीचको अर्थपूर्ण समानताहरू व्यक्त गर्दैनन्।

यस इकाईमा, हामी **News AG** डेटासेटको अन्वेषण जारी राख्नेछौं। सुरु गर्नका लागि, डेटा लोड गरौं र अघिल्लो इकाईबाट केही परिभाषाहरू प्राप्त गरौं।


In [2]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

### एम्बेडिङ भनेको के हो?

**एम्बेडिङ** को विचार भनेको शब्दहरूलाई तिनीहरूको अर्थपूर्ण अर्थ झल्काउने कम-आयामीय घना भेक्टरहरूद्वारा प्रतिनिधित्व गर्नु हो। हामी पछि अर्थपूर्ण शब्द एम्बेडिङ कसरी निर्माण गर्ने भन्ने छलफल गर्नेछौं, तर अहिलेको लागि एम्बेडिङलाई शब्द भेक्टरको आयाम घटाउने तरिका भनेर सोचौं। 

त्यसैले, एम्बेडिङ लेयरले शब्दलाई इनपुटको रूपमा लिन्छ, र निर्दिष्ट गरिएको `embedding_size` को आउटपुट भेक्टर उत्पादन गर्छ। यो केही हदसम्म `Dense` लेयरसँग मिल्दोजुल्दो छ, तर एक-हट इन्कोड गरिएको भेक्टरलाई इनपुटको रूपमा लिने सट्टा, यो शब्द नम्बर लिन सक्षम छ।

हाम्रो नेटवर्कको पहिलो लेयरको रूपमा एम्बेडिङ लेयर प्रयोग गरेर, हामी ब्याग-ऑफ-वर्ड्स मोडेलबाट **एम्बेडिङ ब्याग** मोडेलमा स्विच गर्न सक्छौं, जहाँ हामी पहिलो पटक हाम्रो पाठको प्रत्येक शब्दलाई सम्बन्धित एम्बेडिङमा रूपान्तरण गर्छौं, र त्यसपछि ती सबै एम्बेडिङहरूमा केही समग्र कार्य गणना गर्छौं, जस्तै `sum`, `average` वा `max`।  

![पाँच अनुक्रम शब्दहरूको लागि एम्बेडिङ वर्गीकरण देखाउने छवि।](../../../../../translated_images/ne/embedding-classifier-example.b77f021a7ee67eee.webp)

हाम्रो वर्गीकरण न्युरल नेटवर्क निम्न लेयरहरू समावेश गर्दछ:

* `TextVectorization` लेयर, जसले स्ट्रिङलाई इनपुटको रूपमा लिन्छ, र टोकन नम्बरहरूको टेन्सर उत्पादन गर्छ। हामी केही उपयुक्त शब्दावली आकार `vocab_size` निर्दिष्ट गर्नेछौं, र कम-प्रयोग गरिएका शब्दहरूलाई बेवास्ता गर्नेछौं। इनपुट आकार 1 हुनेछ, र आउटपुट आकार $n$ हुनेछ, किनभने हामी $n$ टोकनहरू परिणामको रूपमा प्राप्त गर्नेछौं, प्रत्येकमा 0 देखि `vocab_size` सम्मका नम्बरहरू समावेश हुनेछन्।
* `Embedding` लेयर, जसले $n$ नम्बरहरू लिन्छ, र प्रत्येक नम्बरलाई दिइएको लम्बाइको घना भेक्टरमा घटाउँछ (हाम्रो उदाहरणमा 100)। यसरी, $n$ आकारको इनपुट टेन्सरलाई $n\times 100$ टेन्सरमा रूपान्तरण गरिनेछ। 
* एग्रिगेसन लेयर, जसले पहिलो अक्षको साथमा यस टेन्सरको औसत लिन्छ, अर्थात् यो विभिन्न शब्दहरूका लागि सबै $n$ इनपुट टेन्सरहरूको औसत गणना गर्नेछ। यो लेयर कार्यान्वयन गर्न, हामी `Lambda` लेयर प्रयोग गर्नेछौं, र यसमा औसत गणना गर्ने कार्य पास गर्नेछौं। आउटपुटको आकार 100 हुनेछ, र यो सम्पूर्ण इनपुट अनुक्रमको संख्यात्मक प्रतिनिधित्व हुनेछ।
* अन्तिम `Dense` रेखीय वर्गीकरणकर्ता।


In [3]:
vocab_size = 30000
batch_size = 128

vectorizer = keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,input_shape=(1,))

model = keras.models.Sequential([
    vectorizer,    
    keras.layers.Embedding(vocab_size,100),
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVec  (None, None)             0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, None, 100)         3000000   
                                                                 
 lambda (Lambda)             (None, 100)               0         
                                                                 
 dense (Dense)               (None, 4)                 404       
                                                                 
Total params: 3,000,404
Trainable params: 3,000,404
Non-trainable params: 0
_________________________________________________________________


`summary` प्रिन्टआउटमा, **output shape** स्तम्भमा पहिलो टेन्सर आयाम `None` मिनिब्याच आकारलाई जनाउँछ, र दोस्रो आयाम टोकन अनुक्रमको लम्बाइलाई जनाउँछ। मिनिब्याचमा रहेका सबै टोकन अनुक्रमहरूको लम्बाइ फरक-फरक हुन्छ। यसलाई कसरी व्यवस्थापन गर्ने भन्ने कुरा हामी अर्को खण्डमा छलफल गर्नेछौं।

अब नेटवर्कलाई प्रशिक्षण गरौं:


In [4]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

print("Training vectorizer")
vectorizer.adapt(ds_train.take(500).map(extract_text))

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
938/938 [==============================] - 20s 20ms/step - loss: 0.7891 - acc: 0.8155 - val_loss: 0.4470 - val_acc: 0.8642


> **नोट** कि हामी डेटा को उपसेटको आधारमा भेक्टराइजर निर्माण गर्दैछौं। यो प्रक्रिया छिटो बनाउनको लागि गरिएको हो, र यसले यस्तो स्थिति निम्त्याउन सक्छ जहाँ हाम्रो पाठका सबै टोकनहरू शब्दकोशमा उपस्थित नहुन सक्छन्। यस्तो अवस्थामा, ती टोकनहरू बेवास्ता गरिनेछ, जसले अलिकति कम शुद्धता परिणाम दिन सक्छ। तर, वास्तविक जीवनमा पाठको उपसेटले प्रायः राम्रो शब्दकोश अनुमान प्रदान गर्दछ।


### भिन्न आकारका भेरिएबल अनुक्रमहरूसँग व्यवहार गर्ने

आउनुहोस्, मिनिब्याचहरूमा प्रशिक्षण कसरी हुन्छ भन्ने कुरा बुझौं। माथिको उदाहरणमा, इनपुट टेन्सरको आयाम 1 छ, र हामी 128-लामो मिनिब्याचहरू प्रयोग गर्छौं, जसले गर्दा टेन्सरको वास्तविक आकार $128 \times 1$ हुन्छ। तर, प्रत्येक वाक्यमा टोकनहरूको संख्या फरक हुन्छ। यदि हामी `TextVectorization` लेयरलाई एकल इनपुटमा लागू गर्छौं भने, टेक्स्ट कसरी टोकनाइज गरिएको छ भन्ने आधारमा फर्काइने टोकनहरूको संख्या फरक हुन्छ:


In [5]:
print(vectorizer('Hello, world!'))
print(vectorizer('I am glad to meet you!'))

tf.Tensor([ 1 45], shape=(2,), dtype=int64)
tf.Tensor([ 112 1271    1    3 1747  158], shape=(6,), dtype=int64)


तर, जब हामी भेक्टराइजरलाई धेरै अनुक्रमहरूमा लागू गर्छौं, यसले आयताकार आकारको टेन्सर उत्पादन गर्नुपर्छ, त्यसैले यसले प्रयोग नगरिएका तत्वहरूलाई PAD टोकन (जसको हाम्रो अवस्थामा शून्य हो) ले भरिन्छ।


In [6]:
vectorizer(['Hello, world!','I am glad to meet you!'])

<tf.Tensor: shape=(2, 6), dtype=int64, numpy=
array([[   1,   45,    0,    0,    0,    0],
       [ 112, 1271,    1,    3, 1747,  158]], dtype=int64)>

यहाँ हामी एम्बेडिङहरू देख्न सक्छौं:


In [7]:
model.layers[1](vectorizer(['Hello, world!','I am glad to meet you!'])).numpy()

array([[[ 1.53059261e-02,  6.80514947e-02,  3.14026810e-02, ...,
         -8.92002955e-02,  1.52911525e-04, -5.65562584e-02],
        [ 2.57456154e-01,  2.79364467e-01, -2.03605562e-01, ...,
         -2.07474351e-01,  8.31158683e-02, -2.03911960e-01],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02]],

       [[ 1.89674050e-01,  2.61548996e-01, -3.67433839e-02, ...,
         -2.07366899e-01, -1.05442435e-01, -2.36952081e-01],
        [ 6.16133213e-02,  1.80511594e-01,  9.77298319e-02, ...,
         -5.46628237e-02, -1.07340455e-01, -1.06589

> **नोट**: प्याडिङको मात्रा कम गर्नको लागि, केही अवस्थामा डेटासेटमा रहेका सबै अनुक्रमहरूलाई बढ्दो लम्बाइको क्रम (वा, अझ स्पष्ट रूपमा, टोकनहरूको संख्या) अनुसार क्रमबद्ध गर्नु उपयुक्त हुन्छ। यसले प्रत्येक मिनिब्याचमा समान लम्बाइका अनुक्रमहरू समावेश हुने सुनिश्चित गर्दछ।


## सेम्यान्टिक एम्बेडिङ: Word2Vec

हाम्रो अघिल्लो उदाहरणमा, एम्बेडिङ लेयरले शब्दहरूलाई भेक्टर प्रतिनिधित्वमा म्याप गर्न सिकेको थियो, तर ती प्रतिनिधित्वहरूमा सेम्यान्टिक अर्थ थिएन। यदि हामीले यस्तो भेक्टर प्रतिनिधित्व सिक्न सक्यौं जसले समान शब्दहरू वा पर्यायवाची शब्दहरूलाई कुनै भेक्टर दूरी (जस्तै युक्लिडियन दूरी) को आधारमा नजिकको भेक्टरहरूसँग मेल खान्छ भने राम्रो हुने थियो।

यसका लागि, हामीले हाम्रो एम्बेडिङ मोडेललाई [Word2Vec](https://en.wikipedia.org/wiki/Word2vec) जस्ता प्रविधि प्रयोग गरेर ठूलो पाठ संग्रहमा प्रि-ट्रेन गर्न आवश्यक छ। यो शब्दहरूको वितरणात्मक प्रतिनिधित्व उत्पादन गर्न प्रयोग गरिने दुई मुख्य आर्किटेक्चरहरूमा आधारित छ:

 - **कन्टिन्युअस ब्याग-अफ-वर्ड्स** (CBoW), जहाँ हामी मोडेललाई वरपरको सन्दर्भबाट शब्दको भविष्यवाणी गर्न प्रशिक्षण दिन्छौं। दिइएको ngram $(W_{-2},W_{-1},W_0,W_1,W_2)$ मा, मोडेलको लक्ष्य $(W_{-2},W_{-1},W_1,W_2)$ बाट $W_0$ को भविष्यवाणी गर्नु हो।
 - **कन्टिन्युअस स्किप-ग्राम** CBoW को विपरीत हो। मोडेलले वरपरको सन्दर्भ शब्दहरूको झ्याल प्रयोग गरेर वर्तमान शब्दको भविष्यवाणी गर्छ।

CBoW छिटो हुन्छ, जबकि स्किप-ग्राम ढिलो भए पनि यो दुर्लभ शब्दहरूको प्रतिनिधित्व गर्न राम्रो काम गर्छ।

![CBoW र स्किप-ग्राम एल्गोरिदमहरूलाई शब्दहरूलाई भेक्टरमा रूपान्तरण गर्न देखाउने छवि।](../../../../../translated_images/ne/example-algorithms-for-converting-words-to-vectors.fbe9207a726922f6.webp)

Google News डेटासेटमा प्रि-ट्रेन गरिएको Word2Vec एम्बेडिङसँग प्रयोग गर्न, हामी **gensim** लाइब्रेरी प्रयोग गर्न सक्छौं। तल हामी 'neural' शब्दसँग सबैभन्दा मिल्दोजुल्दो शब्दहरू फेला पार्छौं।

> **Note:** जब तपाईं पहिलो पटक शब्द भेक्टरहरू सिर्जना गर्नुहुन्छ, तिनीहरू डाउनलोड गर्न केही समय लाग्न सक्छ!


In [8]:
import gensim.downloader as api
w2v = api.load('word2vec-google-news-300')

In [12]:
for w,p in w2v.most_similar('neural'):
    print(f"{w} -> {p}")

neuronal -> 0.7804799675941467
neurons -> 0.7326500415802002
neural_circuits -> 0.7252851724624634
neuron -> 0.7174385190010071
cortical -> 0.6941086649894714
brain_circuitry -> 0.6923246383666992
synaptic -> 0.6699118614196777
neural_circuitry -> 0.6638563275337219
neurochemical -> 0.6555314064025879
neuronal_activity -> 0.6531826257705688


हामी शब्दबाट भेक्टर एम्बेडिङ पनि निकाल्न सक्छौं, जसलाई वर्गीकरण मोडेल प्रशिक्षणमा प्रयोग गर्न सकिन्छ। एम्बेडिङमा ३०० घटकहरू छन्, तर यहाँ स्पष्टताको लागि हामी भेक्टरका पहिलो २० घटकहरू मात्र देखाउँछौं:


In [13]:
w2v['play'][:20]

array([ 0.01226807,  0.06225586,  0.10693359,  0.05810547,  0.23828125,
        0.03686523,  0.05151367, -0.20703125,  0.01989746,  0.10058594,
       -0.03759766, -0.1015625 , -0.15820312, -0.08105469, -0.0390625 ,
       -0.05053711,  0.16015625,  0.2578125 ,  0.10058594, -0.25976562],
      dtype=float32)

महान कुरा सेमान्टिक एम्बेडिङ्सको बारेमा भनेको तपाईंले सेमान्टिक्सको आधारमा भेक्टर इनकोडिङलाई परिमार्जन गर्न सक्नुहुन्छ। उदाहरणका लागि, हामी *राजा* र *महिला* शब्दहरूको भेक्टर प्रतिनिधित्वसँग सकेसम्म नजिक र *पुरुष* शब्दबाट सकेसम्म टाढा हुने शब्द खोज्न अनुरोध गर्न सक्छौं:


In [14]:
w2v.most_similar(positive=['king','woman'],negative=['man'])[0]

('queen', 0.7118192911148071)

उपरोक्त उदाहरणले केही आन्तरिक GenSym जादू प्रयोग गर्दछ, तर आधारभूत तर्क वास्तवमै धेरै सरल छ। एम्बेडिङहरूको बारेमा रोचक कुरा यो हो कि तपाईं एम्बेडिङ भेक्टरहरूमा सामान्य भेक्टर अपरेशनहरू गर्न सक्नुहुन्छ, र त्यसले शब्दको **अर्थहरू**मा अपरेशनहरू प्रतिबिम्बित गर्दछ। उपरोक्त उदाहरणलाई भेक्टर अपरेशनहरूको रूपमा व्यक्त गर्न सकिन्छ: हामी **KING-MAN+WOMAN** सँग सम्बन्धित भेक्टर गणना गर्छौं (सम्बन्धित शब्दहरूको भेक्टर प्रतिनिधित्वमा `+` र `-` अपरेशनहरू गरिन्छ), र त्यसपछि उक्त भेक्टरसँग नजिकको शब्द शब्दकोशमा फेला पार्छौं:


In [15]:
# get the vector corresponding to kind-man+woman
qvec = w2v['king']-1.7*w2v['man']+1.7*w2v['woman']
# find the index of the closest embedding vector 
d = np.sum((w2v.vectors-qvec)**2,axis=1)
min_idx = np.argmin(d)
# find the corresponding word
w2v.index_to_key[min_idx]

'queen'

> **NOTE**: हामीले *man* र *woman* भेक्टरहरूमा सानो गुणांक थप्नुपरेको थियो - यसलाई हटाएर के हुन्छ हेर्न प्रयास गर्नुहोस्।

निकटतम भेक्टर पत्ता लगाउन, हामीले TensorFlow को मेकानिजम प्रयोग गरेर हाम्रो भेक्टर र शब्दकोशमा रहेका सबै भेक्टरहरू बीचको दूरीको भेक्टर गणना गर्छौं, र त्यसपछि `argmin` प्रयोग गरेर न्यूनतम शब्दको सूचकांक पत्ता लगाउँछौं।


Word2Vec शब्दार्थलाई व्यक्त गर्नको लागि एक उत्कृष्ट तरिका जस्तो देखिन्छ, तर यसमा निम्न जस्ता धेरै कमजोरीहरू छन्:

* CBoW र skip-gram मोडेलहरू **predictive embeddings** हुन्, र यीले केवल स्थानीय सन्दर्भलाई मात्र ध्यानमा राख्छन्। Word2Vec ले विश्वव्यापी सन्दर्भको फाइदा उठाउँदैन।
* Word2Vec ले शब्दको **morphology** लाई ध्यानमा राख्दैन, अर्थात् शब्दको अर्थ शब्दका विभिन्न भागहरू, जस्तै मूल (root), मा निर्भर हुन सक्छ भन्ने तथ्यलाई बेवास्ता गर्छ।  

**FastText** ले दोस्रो सीमालाई पार गर्न प्रयास गर्छ, र Word2Vec मा आधारित भएर प्रत्येक शब्द र त्यस शब्दभित्र पाइने character n-grams का लागि भेक्टर प्रतिनिधित्वहरू सिक्छ। यी प्रतिनिधित्वहरूको मानहरूलाई प्रत्येक प्रशिक्षण चरणमा एक भेक्टरमा औसत गरिन्छ। यसले पूर्व-प्रशिक्षणमा धेरै अतिरिक्त गणना थपे पनि, यसले शब्द embeddings लाई उप-शब्द जानकारीलाई समेट्न सक्षम बनाउँछ।

अर्को विधि, **GloVe**, शब्द embeddings को लागि फरक दृष्टिकोण प्रयोग गर्छ, जुन word-context matrix को factorization मा आधारित छ। सुरुमा, यो एउटा ठूलो म्याट्रिक्स बनाउँछ जसले विभिन्न सन्दर्भहरूमा शब्दहरूको उपस्थितिको संख्या गन्छ, र त्यसपछि यो म्याट्रिक्सलाई कम आयाममा प्रतिनिधित्व गर्न प्रयास गर्छ जसले पुनर्निर्माण हानिलाई न्यूनतम बनाउँछ।

gensim पुस्तकालयले ती शब्द embeddings लाई समर्थन गर्छ, र तपाईंले माथिको मोडेल लोड गर्ने कोड परिवर्तन गरेर तिनीहरूसँग प्रयोग गर्न सक्नुहुन्छ।


## Keras मा pretrained embeddings प्रयोग गर्ने

हामी माथिको उदाहरणलाई संशोधन गरेर हाम्रो embedding layer मा semantic embeddings, जस्तै Word2Vec, प्रयोग गर्न सक्दछौं। pretrained embedding र text corpus को शब्दावलीहरू प्रायः मेल नखाने सम्भावना हुन्छ, त्यसैले हामीलाई एउटा विकल्प छान्नुपर्छ। यहाँ हामी दुई सम्भावित विकल्पहरू अन्वेषण गर्छौं: tokenizer vocabulary प्रयोग गर्ने, र Word2Vec embeddings को vocabulary प्रयोग गर्ने।

### Tokenizer vocabulary प्रयोग गर्ने

Tokenizer vocabulary प्रयोग गर्दा, vocabulary का केही शब्दहरूको Word2Vec embeddings हुनेछ, र केही हराउनेछन्। हाम्रो vocabulary size `vocab_size` छ, र Word2Vec embedding vector को लम्बाइ `embed_size` छ भने, embedding layer को तौल म्याट्रिक्सको आकार `vocab_size`$\times$`embed_size` हुनेछ। हामी यो म्याट्रिक्सलाई vocabulary मार्फत गएर भरनेछौं:


In [9]:
embed_size = len(w2v.get_vector('hello'))
print(f'Embedding size: {embed_size}')

vocab = vectorizer.get_vocabulary()
W = np.zeros((vocab_size,embed_size))
print('Populating matrix, this will take some time...',end='')
found, not_found = 0,0
for i,w in enumerate(vocab):
    try:
        W[i] = w2v.get_vector(w)
        found+=1
    except:
        # W[i] = np.random.normal(0.0,0.3,size=(embed_size,))
        not_found+=1

print(f"Done, found {found} words, {not_found} words missing")

Embedding size: 300
Populating matrix, this will take some time...Done, found 4551 words, 784 words missing


शब्दहरू जुन Word2Vec शब्दकोशमा उपलब्ध छैनन्, हामी तिनीहरूलाई शून्यको रूपमा छोड्न सक्छौं, वा एउटा र्यान्डम भेक्टर सिर्जना गर्न सक्छौं।

अब हामी प्रीट्रेन गरिएको वजनहरू सहितको एम्बेडिङ लेयर परिभाषित गर्न सक्छौं:


In [10]:
emb = keras.layers.Embedding(vocab_size,embed_size,weights=[W],trainable=False)
model = keras.models.Sequential([
    vectorizer, emb,
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])

In [11]:
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),
          validation_data=ds_test.map(tupelize).batch(batch_size))

938/938 [==============================] - 10s 10ms/step - loss: 1.1075 - acc: 0.7822 - val_loss: 0.9134 - val_acc: 0.8175


> **Note**: हामीले `Embedding` सिर्जना गर्दा `trainable=False` सेट गरेको देखिन्छ, जसको अर्थ हामी Embedding लेयरलाई पुनः प्रशिक्षण गरिरहेका छैनौं। यसले सटीकता अलि कम हुन सक्छ, तर प्रशिक्षणको गति बढाउँछ।

### Embedding शब्दकोश प्रयोग गर्दै

अघिल्लो विधिको एउटा समस्या भनेको TextVectorization र Embedding मा प्रयोग गरिएका शब्दकोशहरू फरक हुनु हो। यस समस्यालाई समाधान गर्न, हामी निम्न विकल्पहरू प्रयोग गर्न सक्छौं:
* हाम्रो शब्दकोशमा Word2Vec मोडेल पुनः प्रशिक्षण गर्नुहोस्।
* Pretrained Word2Vec मोडेलको शब्दकोशबाट हाम्रो डेटासेट लोड गर्नुहोस्। डेटासेट लोड गर्दा प्रयोग गरिने शब्दकोश लोड गर्ने क्रममा निर्दिष्ट गर्न सकिन्छ।

दोस्रो विधि सजिलो देखिन्छ, त्यसैले यसलाई कार्यान्वयन गरौं। सबैभन्दा पहिले, हामी Word2Vec embeddings बाट लिइएको निर्दिष्ट शब्दकोशसहित `TextVectorization` लेयर सिर्जना गर्नेछौं:


In [12]:
vocab = list(w2v.vocab.keys())
vectorizer = keras.layers.experimental.preprocessing.TextVectorization(input_shape=(1,))
vectorizer.set_vocabulary(vocab)

Gensim शब्द एम्बेडिङ्स पुस्तकालयमा एउटा सुविधाजनक फङ्सन, `get_keras_embeddings`, समावेश छ, जसले तपाईंको लागि स्वचालित रूपमा सम्बन्धित Keras एम्बेडिङ्स तह सिर्जना गर्नेछ।


In [13]:
model = keras.models.Sequential([
    vectorizer, 
    w2v.get_keras_embedding(train_embeddings=False),
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128),epochs=5)

Epoch 1/5
938/938 [==============================] - 20s 14ms/step - loss: 1.3377 - acc: 0.4978 - val_loss: 1.2995 - val_acc: 0.5647
Epoch 2/5
938/938 [==============================] - 10s 10ms/step - loss: 1.2587 - acc: 0.5722 - val_loss: 1.2339 - val_acc: 0.5842
Epoch 3/5
938/938 [==============================] - 10s 10ms/step - loss: 1.1980 - acc: 0.5884 - val_loss: 1.1826 - val_acc: 0.5954
Epoch 4/5
938/938 [==============================] - 12s 13ms/step - loss: 1.1503 - acc: 0.6002 - val_loss: 1.1417 - val_acc: 0.6018
Epoch 5/5
938/938 [==============================] - 11s 12ms/step - loss: 1.1120 - acc: 0.6097 - val_loss: 1.1083 - val_acc: 0.6104


हामीले उच्च सटीकता देख्न नसक्नुको एक कारण भनेको हाम्रो डाटासेटका केही शब्दहरू प्रि-ट्रेन गरिएको GloVe शब्दकोशमा नभएका कारण हो, जसले गर्दा ती शब्दहरूलाई मूलतः बेवास्ता गरिन्छ। यसलाई समाधान गर्नका लागि, हामी हाम्रो डाटासेटमा आधारित आफ्नै एम्बेडिङहरू प्रशिक्षण गर्न सक्छौं।


## सन्दर्भात्मक एम्बेडिङहरू

पारम्परिक प्रीट्रेन गरिएको एम्बेडिङ प्रतिनिधित्वहरू जस्तै Word2Vec को मुख्य सीमितता यो हो कि, यद्यपि तिनीहरूले शब्दको केही अर्थ समात्न सक्छन्, तिनीहरूले विभिन्न अर्थहरू बीच फरक छुट्याउन सक्दैनन्। यसले डाउनस्ट्रीम मोडेलहरूमा समस्या उत्पन्न गर्न सक्छ।

उदाहरणका लागि, 'play' शब्दको यी दुई वाक्यहरूमा फरक अर्थ छ:
- म थिएटरमा एउटा **play** हेर्न गएँ।
- जोन आफ्ना साथीहरूसँग **play** गर्न चाहन्छ।

हामीले चर्चा गरेका प्रीट्रेन गरिएको एम्बेडिङहरूले 'play' शब्दको दुवै अर्थलाई एउटै एम्बेडिङमा प्रतिनिधित्व गर्छ। यस सीमिततालाई पार गर्नका लागि, हामीलाई **भाषा मोडेल** मा आधारित एम्बेडिङहरू निर्माण गर्न आवश्यक छ, जुन ठूलो पाठको संग्रहमा प्रशिक्षित गरिएको हुन्छ, र *जान्दछ* कि शब्दहरू विभिन्न सन्दर्भहरूमा कसरी राख्न सकिन्छ। सन्दर्भात्मक एम्बेडिङहरूको चर्चा यस ट्युटोरियलको दायराभन्दा बाहिर छ, तर हामी यसबारे भाषा मोडेलहरूको कुरा गर्दा अर्को युनिटमा फर्कनेछौं।



---

**अस्वीकरण**:  
यो दस्तावेज़ AI अनुवाद सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) प्रयोग गरेर अनुवाद गरिएको छ। हामी शुद्धताको लागि प्रयास गर्छौं, तर कृपया ध्यान दिनुहोस् कि स्वचालित अनुवादमा त्रुटिहरू वा अशुद्धताहरू हुन सक्छ। यसको मूल भाषा मा रहेको मूल दस्तावेज़लाई आधिकारिक स्रोत मानिनुपर्छ। महत्वपूर्ण जानकारीको लागि, व्यावसायिक मानव अनुवाद सिफारिस गरिन्छ। यस अनुवादको प्रयोगबाट उत्पन्न हुने कुनै पनि गलतफहमी वा गलत व्याख्याको लागि हामी जिम्मेवार हुने छैनौं।
